In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [2]:
import ee 
import geemap

ee.Initialize(project="ee-india-reservoirs")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [4]:
from src.et_blue.exporting_utils import process_et_blue
from src.et_blue.compute_et_blue import compute_et_blue

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
)

from utils.date_utils import print_collection_dates

---

## Constants

In [6]:
YEAR = 2018

PATH_TO_AOI = "projects/thurgau-irrigation/assets/FribourgAndVaud/broye_bounding_box"
PATH_TO_ET_PRODUCT = (
    f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_products/decadal_Landsat_10m"
)
PATH_TO_ET_GREEN = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_dekadal_from_Landsat_30m_v2"

ET_GREEN_BAND_NAME = "ET_green"
ET_BAND_NAME = "ET"
SCALING_FACTOR = 100
DYNAMIC = True
SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
NUMBER_OF_IMAGES = 21
TEMPORAL_RESOLUTION = "dekadal"


# ET_BLUE_RAW_PATH = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_raw/ET_blue_raw_{YEAR}_{TEMPORAL_RESOLUTION}_from_Landsat_30m"
ET_BLUE_RAW_PATH = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_raw/ET_blue_raw_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v2"
print(ET_BLUE_RAW_PATH)
EXPORT_IMAGE_RESOLUTION = 30  # in meters

projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_raw/ET_blue_raw_dekadal_from_Landsat_30m_v2


## 1. Load Assets

In [7]:
aoi = ee.FeatureCollection(PATH_TO_AOI).geometry().simplify(500).buffer(100)

In [8]:
et_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT)
    # .filter(ee.Filter.eq("Region", "Thurgau"))
    .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
    .map(
        lambda img: back_to_float(
            img, SCALING_FACTOR, DYNAMIC, SCALING_FACTOR_PROPERTY_NAME
        )
    )
).sort("system:time_start")

et_collection_list = et_collection.toList(et_collection.size())


# Sanity checks:
print_collection_dates(et_collection)

Dates of images in the collection:
2018-04-01
2018-04-11
2018-04-21
2018-05-01
2018-05-11
2018-05-21
2018-06-01
2018-06-11
2018-06-21
2018-07-01
2018-07-11
2018-07-21
2018-08-01
2018-08-11
2018-08-21
2018-09-01
2018-09-11
2018-09-21
2018-10-01
2018-10-11
2018-10-21


In [9]:
et_green_collection = (
    ee.ImageCollection(PATH_TO_ET_GREEN)
    .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
    .map(lambda img: back_to_float(img, SCALING_FACTOR))
)

et_green_list = et_green_collection.toList(et_collection.size())

# Sanity checks:
print_collection_dates(et_green_collection)
# print_value_ranges(et_green_collection, ET_GREEN_BAND_NAME)

Dates of images in the collection:
2018-04-01
2018-04-11
2018-04-21
2018-05-01
2018-05-11
2018-05-21
2018-06-01
2018-06-11
2018-06-21
2018-07-01
2018-07-11
2018-07-21
2018-08-01
2018-08-11
2018-08-21
2018-09-01
2018-09-11
2018-09-21
2018-10-01
2018-10-11
2018-10-21


## 2. Compute and Export ET_blue raw

In [16]:
process_et_blue(
    et_collection_list=et_collection_list,
    et_green_list=et_green_list,
    year=YEAR,
    aoi=aoi,
    asset_path=ET_BLUE_RAW_PATH,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=EXPORT_IMAGE_RESOLUTION,
)

Exporting ET_blue_raw_dekadal_2019_04_D1 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_raw/ET_blue_raw_2019_dekadal_from_Landsat_30m_v2/ET_blue_raw_dekadal_2019_04_D1
Exporting ET_blue_raw_dekadal_2019_04_D2 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_raw/ET_blue_raw_2019_dekadal_from_Landsat_30m_v2/ET_blue_raw_dekadal_2019_04_D2
Exporting ET_blue_raw_dekadal_2019_04_D3 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_raw/ET_blue_raw_2019_dekadal_from_Landsat_30m_v2/ET_blue_raw_dekadal_2019_04_D3
Exporting ET_blue_raw_dekadal_2019_05_D1 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_raw/ET_blue_raw_2019_dekadal_from_Landsat_30m_v2/ET_blue_raw_dekadal_2019_05_D1
Exporting ET_blue_raw_dekadal_2019_05_D2 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_raw/ET_blue_raw_2019_dekadal_from_Landsat_30m_v2/ET_blue_raw_dekadal_2019_05_D2
Exporting ET_blue_raw_dekadal_2019_05_D3

In [ ]:
et_blue_raw = []

for i in range(NUMBER_OF_IMAGES):
    # Process ET images
    et_image = ee.Image(et_collection_list.get(i))
    et_green = ee.Image(et_green_list.get(i))
    et_blue = compute_et_blue(et_image, et_green)
    et_blue_raw.append(et_blue)

In [ ]:
ee.Image(et_blue_raw[0]).bandNames().getInfo()

In [ ]:
Map = geemap.Map(height="800px")

image = ee.Image(et_blue_raw[2])
# image_green = ee.Image(et_green_list.get(20))
# image_et = ee.Image(et_collection_list.get(20))
# image_ET = ee.Image(et_collection_list.get(17))

vis_params = {
    "bands": ["ET_blue"],
    "min": 0,
    "max": 1,
    "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
}

# vis_green = {
#     "bands": ["ET_green"],
#     "min": 0,
#     "max": 3,
#     "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
# }

# vis_ET = {
#     "bands": ["downscaled"],
#     "min": 0,
#     "max": 5,
#     "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
# }


Map.center_object(aoi, 12)
Map.addLayer(image, vis_params, "ET blue raw")
Map.add_colorbar(vis_params, label="ET blue raw [mm/month]")
Map.center_object(aoi, 12)
# Map.addLayer(image_green, vis_green, "ET green")
# Map.add_colorbar(vis_green, label="ET green [mm/month]")
# Map.addLayer(image_et, vis_ET, "ET")
# Map.add_colorbar(vis_ET, label="ET [mm/month]")


Map

## Sanity check

In [6]:
et_blue_raw_check = ee.ImageCollection("projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_raw/ET_blue_raw_2018_dekadal_from_Landsat_30m").map(lambda img: back_to_float(img, SCALING_FACTOR))

print_collection_dates(et_blue_raw_check)

Dates of images in the collection:
2018-04-01
2018-04-11
2018-04-21
2018-05-01
2018-05-11
2018-05-21
2018-06-01
2018-06-11
2018-06-21
2018-07-01
2018-07-11
2018-07-21
2018-08-01
2018-08-11
2018-08-21
2018-09-01
2018-09-11
2018-09-21
2018-10-01
2018-10-11
2018-10-21


In [7]:
et_blue_raw_check_list = et_blue_raw_check.toList(et_blue_raw_check.size())

In [12]:
image_7 = ee.Image(et_blue_raw_check_list.get(4))

Map = geemap.Map(height="800px")

vis_params = {

    "bands": ["ET_blue"],
    "min": 0,
    "max": 1,
    "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
}

Map.center_object(aoi, 12)
Map.addLayer(image_7, vis_params, "ET blue raw")
Map.add_colorbar(vis_params, label="ET blue raw [mm/month]")
Map

Map(center=[46.78675332740657, 6.874554339425382], controls=(WidgetControl(options=['position', 'transparent_b…